In [2]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time
from matplotlib import image
from torchvision import transforms, datasets, models
from efficientnet_pytorch import EfficientNet
import time
import timm
import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_data  = pd.read_csv(r'/opt/ml/input/data/train/new_train.csv')

In [3]:
train_data

,Full Path,NewClass,-,Class Compare,Original Class,-.1,Mask,Gender,Age,-.2,ImageName,Gender.1,Age.1,ID
0,/opt/ml/input/data/train/images/000001_female_...,10,NaN,True,10,NaN,1,1,1,NaN,incorrect_mask.jpg,female,45,1
1,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask1.jpg,female,45,1
2,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask2.jpg,female,45,1
3,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask3.jpg,female,45,1
4,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask4.jpg,female,45,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,/opt/ml/input/data/train/images/006959_male_As...,0,NaN,True,0,NaN,0,0,0,NaN,mask2.jpg,male,19,6959
18896,/opt/ml/input/data/train/images/006959_male_As...,0,NaN,True,0,NaN,0,0,0,NaN,mask3.jpg,male,19,6959
18897,/opt/ml/input/data/train/images/006959_male_As...,0,NaN,True,0,NaN,0,0,0,NaN,mask4.jpg,male,19,6959
18898,/opt/ml/input/data/train/images/006959_male_As...,0,NaN,True,0,NaN,0,0,0,NaN,mask5.jpg,male,19,6959


## DataSet 형성

In [4]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
class imgDataset(Dataset):
    def __init__(self, csv_path):
        #self.to_tensor = transforms.ToTensor()
        self.main_path = csv_path
        self.total_data = pd.read_csv(self.main_path + '/' + 'new_train.csv')
        #self.image_folder = (self.main_path + '/' + 'images')
        self.transform = transforms.Compose([
                                            transforms.CenterCrop(384),
                                            transforms.Resize(256),
                                            transforms.ToTensor(),
                                             
                                            transforms.Normalize(mean = [0.5,0.5,0.5],
                                                                 std =[0.5,0.5,0.5])])#384 384로 crop을 하고, 그런 후 256으로 resize
        #self.path = r'C:\Users\harry\Desktop\Naver\train\train\images'
    #def __len__(self):
   #     return 7 * self.human_data.shape[0]
    def __getitem__(self,index):
        image = Image.open(self.total_data.iloc[index]['Full Path'])
        image = self.transform(image)
        class_num = self.total_data.iloc[index]['NewClass']
        return class_num, image
    def __len__(self):
        return self.total_data.shape[0]

In [5]:
class_img = imgDataset(r'/opt/ml/input/data/train')

In [6]:
len(class_img)

18900

In [7]:
k, j = class_img[18899]

In [8]:
print(k)

12


In [9]:
print(j)

tensor([[[0.6706, 0.6706, 0.6706,  ..., 0.5922, 0.5922, 0.5922],
         [0.6706, 0.6706, 0.6706,  ..., 0.5922, 0.5922, 0.5922],
         [0.6784, 0.6784, 0.6784,  ..., 0.5922, 0.5922, 0.5922],
         ...,
         [0.8275, 0.8353, 0.8588,  ..., 0.6471, 0.6471, 0.6627],
         [0.8275, 0.8431, 0.8667,  ..., 0.6549, 0.6549, 0.6706],
         [0.8275, 0.8510, 0.8745,  ..., 0.6706, 0.6627, 0.6784]],

        [[0.6784, 0.6784, 0.6784,  ..., 0.6000, 0.6000, 0.6000],
         [0.6784, 0.6784, 0.6784,  ..., 0.6000, 0.6000, 0.6000],
         [0.6863, 0.6863, 0.6863,  ..., 0.6000, 0.6000, 0.6000],
         ...,
         [0.8667, 0.8667, 0.8510,  ..., 0.6235, 0.6235, 0.6392],
         [0.8667, 0.8745, 0.8667,  ..., 0.6314, 0.6314, 0.6471],
         [0.8667, 0.8824, 0.8667,  ..., 0.6471, 0.6392, 0.6549]],

        [[0.6392, 0.6392, 0.6392,  ..., 0.5529, 0.5529, 0.5529],
         [0.6392, 0.6392, 0.6392,  ..., 0.5529, 0.5529, 0.5529],
         [0.6471, 0.6471, 0.6471,  ..., 0.5529, 0.5529, 0.

## Train Valid 분리

In [10]:
train_len = int(len(class_img) * 0.8)
print(train_len)
val_len = len(class_img) - train_len
print(val_len)
from torch.utils.data.dataset import random_split
train_dataset, val_dataset = random_split(class_img, [train_len,val_len])

15120
3780


## DataLoader 형성

In [11]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset = train_dataset, batch_size=64, shuffle = True, drop_last=True)    #통상 첨에 시작할때 batch는 64에서 128로 잡는다  
val_loader = DataLoader(dataset = val_dataset, batch_size=64, shuffle = True, drop_last=True)

## Pretrained 모델(다중 택일) + MLP 형성

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#model = Model().to(device)
#loss_func = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

cuda:0


In [13]:
def getModel(model_name):
    # PyTorch에서 제공하는 ResNet을 불러오고, 마지막 FC layer를 수정하는 방법을 살펴보겠습니다.
    if model_name == 'resnet152':
        model = models.resnet152(pretrained=True)
        num_features = model.fc.in_features
        #for param in model.parameters():
        #    param.requires_grad = False
        model.fc = nn.Linear(num_features, 18)
    elif model_name == 'resnet34':
        model = models.resnet34(pretrained=True)
        num_features = model.fc.in_features
        #for param in model.parameters():
        #    param.requires_grad = False
        model.fc = nn.Linear(num_features, 18)
    elif model_name == 'EfficientNet':
        model = EfficientNet.from_pretrained('efficientnet-b0', num_classes = 18)
        #for n, p in model.named_parameters():
        #     if '_fc' not in n:
        #        p.requires_grad = False
        #        model = torch.nn.parallel.DistributedDataParallel(model)
    elif model_name == 'vit_base_patch16_224':
        model =  timm.create_model('vit_base_patch16_224',pretrained=True, num_classes = 18)

        #for param in model.parameters():
        #    param.requires_grad = False
 
    model = model.to(device)

    #criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    #optimizer = optim.SGD(model.parameters(), lr=lr)
    
    return model

## Model을 넣었을때, train data에 대한 성능, valid data에 대한 성능을 구해주고, epoch마다 model을 저장해 주는 함수

In [14]:
def Train_and_Valid_check(model, model_name):
    from tqdm.notebook import tqdm
    MODEL_PATH ="models_saved"
    
    num_epochs = 10
    #start_time = time.time()

    for epoch in range(num_epochs):
        """ Training Phase """
        model.train()

        running_loss = 0.
        running_corrects = 0

        # load a batch data of images
        for labels, img in tqdm(train_loader):
            #inputs = inputs.to(device)
            labels = labels.to(device)#label이 class
            img = img.to(device)#img가 

            # forward inputs and get output
            optimizer.zero_grad()
            outputs = model(img)
            _, preds = torch.max(outputs, 1)
            loss = loss_func(outputs, labels)

            # get loss value and update the network weights
            loss.backward()
            optimizer.step()
            #print(img.size(0))
            running_loss += loss.item() * img.size(0)#64개 만큼의 loss가 나온다 
            # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
        #딕셔너리.items()와 다른것 주의!!
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects / len(train_dataset) * 100.
        print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% '.format(epoch, epoch_loss, epoch_acc))


        model.eval()

        with torch.no_grad():
            running_loss = 0.
            running_corrects = 0

            for labels, inputs in tqdm(val_loader):#얼마나 진행됐고 얼마나 남았는지 알려줌
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_func(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(val_dataset)
            epoch_acc = running_corrects / len(val_dataset) * 100.
            #wandb.log({"val_acc":epoch_acc, "val_loss":epoch_loss})
            print('[Validation #{}] Loss: {:.4f} Acc: {:.4f}% '.format(epoch, epoch_loss, epoch_acc))
        if not os.path.exists(MODEL_PATH):
            os.makedirs(MODEL_PATH)
        torch.save(model, os.path.join(MODEL_PATH, f"{model_name}_is_model_and_{epoch}th_epoch.pt"))
        #os.path.join(MODEL_PATH, "hand_mademodel_with_new_csv.pt"

## 모델 설정(resnet152), lossfunction 설정, optimizer 설정

In [15]:
model = getModel('resnet152')
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Selecting Pretrained Model(resnet152 성능 check)

In [16]:
Train_and_Valid_check(model, 'resnet152')


[Train #0] Loss: 1.0951 Acc: 67.1958% 



[Validation #0] Loss: 0.4401 Acc: 86.7460% 



[Train #1] Loss: 0.3194 Acc: 89.5370% 



[Validation #1] Loss: 0.2525 Acc: 91.6667% 



[Train #2] Loss: 0.1573 Acc: 95.4365% 


...


[Validation #7] Loss: 0.0878 Acc: 97.3016% 



[Train #8] Loss: 0.0104 Acc: 99.8016% 



[Validation #8] Loss: 0.1602 Acc: 94.6296% 



[Train #9] Loss: 0.0088 Acc: 99.7950% 



[Validation #9] Loss: 0.0782 Acc: 97.4074% 


## 모델 설정(resnet34), lossfunction 설정, optimizer 설정

In [17]:
model = getModel('resnet34')
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)

## Selecting Pretrained Model(resnet34 성능 check)

In [18]:
Train_and_Valid_check(model,'resnet34')


[Train #0] Loss: 0.9554 Acc: 71.4616% 



[Validation #0] Loss: 0.4074 Acc: 86.8254% 



[Train #1] Loss: 0.3052 Acc: 89.7619% 



[Validation #1] Loss: 0.2800 Acc: 89.9735% 



[Train #2] Loss: 0.1735 Acc: 94.3651% 


...


[Validation #7] Loss: 0.0983 Acc: 96.1640% 



[Train #8] Loss: 0.0103 Acc: 99.8413% 



[Validation #8] Loss: 0.0937 Acc: 96.6931% 



[Train #9] Loss: 0.0088 Acc: 99.8280% 



[Validation #9] Loss: 0.0911 Acc: 96.5608% 


## 모델설정(EfficientNet), lossfunction 설정, optimizer 설정

In [19]:

model = getModel('EfficientNet')
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)


Loaded pretrained weights for efficientnet-b0


## Selecting Pretrained Model(EfficientNet 성능 check)

In [20]:
Train_and_Valid_check(model,'EfficientNet')


[Train #0] Loss: 1.9045 Acc: 48.3399% 



[Validation #0] Loss: 1.3194 Acc: 62.8042% 



[Train #1] Loss: 1.0970 Acc: 68.3730% 



[Validation #1] Loss: 0.8321 Acc: 76.0847% 



[Train #2] Loss: 0.7839 Acc: 77.5860% 


...


[Validation #7] Loss: 0.3142 Acc: 88.8624% 



[Train #8] Loss: 0.3155 Acc: 89.3188% 



[Validation #8] Loss: 0.2899 Acc: 89.8148% 



[Train #9] Loss: 0.2834 Acc: 90.3770% 



[Validation #9] Loss: 0.2683 Acc: 90.2116% 


## 가장 좋았던 모델 불러오기

In [3]:
model = torch.load(os.path.join("models_saved", "resnet152_is_model_and_9th_epoch.pt"))

## Test Data Set정의

In [4]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [6]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [8]:
from torch.utils.data import DataLoader
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
'''
self.transform = transforms.Compose([
                                            transforms.CenterCrop(384),
                                            transforms.Resize(256),
                                            transforms.ToTensor(),
                                             
                                            transforms.Normalize(mean = [0.5,0.5,0.5],
                                                                 std =[0.5,0.5,0.5])])#384 384로 crop을 하고, 그런 후 256으로 resize
'''
transform = transforms.Compose([
                    transforms.CenterCrop(384),
                    transforms.Resize(256),
                    transforms.ToTensor(),
                                             
                    transforms.Normalize(mean = [0.5,0.5,0.5],std =[0.5,0.5,0.5])
                                ])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
#model = MyModel(num_classes=18).to(device)
#model = torch.load(os.path.join(MODEL_PATH, "model_pickle.pt"))
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
